In [ ]:
!pip install jax jaxlib
!pip install --quiet --upgrade scipy
!pip install --quiet jax jaxlib optax

In [ ]:
import jax
from jax.scipy.stats import norm
import jax.numpy as jnp
from scipy.stats import norm
import time
import jax.numpy as jnp
from numpy.polynomial.legendre import leggauss
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar, brentq, minimize
from scipy.special import gamma
from numpy.polynomial.legendre import leggauss
import warnings
warnings.filterwarnings('ignore')
from numpy.random import default_rng
from math import log
from numpy.random import default_rng, SeedSequence
from scipy.stats import kstwobign, cramervonmises, uniform
from joblib import Parallel, delayed
from itertools import zip_longest
from collections import OrderedDict


# ==============================================================
# ETELN Simulation Same J
# ==============================================================


In [ ]:
# ==============================================================
# ETELN ARE Table
# ==============================================================

# ---------- pretty, wide table printing ----------
pd.set_option("display.width", 2000)
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)


class ETELNARE:
    def __init__(self, theta=1.0, kuma_a=1.0, kuma_b=1.0, n_quad=200):
        self.theta = theta
        self.kuma_a = kuma_a
        self.kuma_b = kuma_b
        self.n_quad = n_quad
        # Precompute Gauss–Legendre nodes & weights
        self.nodes, self.weights = np.polynomial.legendre.leggauss(n_quad)
        # Transform nodes from [-1,1] → [0,1]
        self.u = 0.5 * (self.nodes + 1)
        self.w = 0.5 * self.weights

    def kumaraswamy_weight(self, u, a=None, b=None):
        if a is None: a = self.kuma_a
        if b is None: b = self.kuma_b
        return a * b * (u ** (a - 1)) * ((1 - u**a) ** (b - 1))

    # ---- Dk integral using Gauss–Legendre ----
    def compute_dk(self, beta, k):
        """
        Compute d_{h,k}(β) = ∫₀¹ J(u) [Φ⁻¹[(2^β-(2^β-1)u)^(-1/β)]]^k du
        """
        u, w = self.u, self.w
        weights = self.kumaraswamy_weight(u)

        # u_0(β) = [2^β - (2^β-1)u]^(-1/β)
        base = 2**beta - (2**beta - 1) * u

        # Ensure base > 0 and compute u0
        mask = base > 0
        u0 = np.zeros_like(u)
        u0[mask] = base[mask] ** (-1/beta)

        # Clip u0 to valid probability range [ε, 1-ε]
        eps = 1e-9
        u0 = np.clip(u0, eps, 1 - eps)

        # ξ = Φ⁻¹[u₀(β)]
        xi = norm.ppf(u0)

        # Integrand: J(u) * ξ^k
        integrand = weights * (xi ** k)

        return np.sum(w * integrand)

    def tau(self, beta):
        """τ_h(β) = d_{h,2}(β) - [d_{h,1}(β)]²"""
        D1 = self.compute_dk(beta, 1)
        D2 = self.compute_dk(beta, 2)
        return D2 - D1**2

    def solve_beta(self, mu1, mu2, beta_range=(-3, 3)):
        """
        Solve implicit equation for β, then compute α
        For ETELN: μ₁ = log(θ) + (1/α)d_{h,1}
        So: d_{h,1}/α = μ₁ - log(θ)

        Implicit equation: d_{h,1}/√τ_h = (μ₁ - log θ)/√Δ_h
        """
        Delta = mu2 - mu1**2

        if Delta <= 0:
            print(f"  Warning: Delta <= 0 ({Delta:.6f})")
            return np.nan, np.nan

        def f(beta):
            try:
                D1 = self.compute_dk(beta, 1)
                t = self.tau(beta)

                if t <= 0:
                    return np.nan

                # R(β) = d_{h,1}/√τ_h - (μ₁ - log θ)/√Δ_h
                # Note: For ETELN, it's (μ₁ - log θ) not (log θ - μ₁)
                lhs = D1 / np.sqrt(t)
                rhs = (mu1 - np.log(self.theta)) / np.sqrt(Delta)

                return lhs - rhs
            except:
                return np.nan

        # Try to find sign change
        beta_test = np.linspace(beta_range[0], beta_range[1], 20)
        f_vals = [f(b) for b in beta_test]
        f_vals = np.array([v if not np.isnan(v) else 0 for v in f_vals])

        # Check for sign change
        sign_changes = np.where(np.diff(np.sign(f_vals)))[0]

        if len(sign_changes) == 0:
            return np.nan, np.nan

        # Use first sign change
        idx = sign_changes[0]
        beta_low = beta_test[idx]
        beta_high = beta_test[idx + 1]

        from scipy.optimize import brentq
        try:
            beta_hat = brentq(f, beta_low, beta_high, xtol=1e-10, rtol=1e-8, maxiter=500)
            t = self.tau(beta_hat)
            alpha_hat = np.sqrt(t / Delta)
            return alpha_hat, beta_hat
        except Exception as e:
            print(f"  Brentq failed: {e}")
            return np.nan, np.nan

    # ---- Variance–covariance matrix via Λ integrals ----
    def compute_lambda_integral(self, type_num, alpha, beta):
        """
        Compute Λ_{h,i} integrals for ETELN
        These involve 1/φ(ξ) terms
        """
        u, w = self.u, self.w
        weights = self.kumaraswamy_weight(u)

        # u_0(β) = [2^β - (2^β-1)u]^(-1/β)
        base = 2**beta - (2**beta - 1) * u
        mask = base > 0
        u0 = np.zeros_like(u)
        u0[mask] = base[mask] ** (-1/beta)

        # Clip to valid range
        eps = 1e-9
        u0 = np.clip(u0, eps, 1 - eps)

        # ξ = Φ⁻¹[u₀(β)]
        xi = norm.ppf(u0)

        # φ(ξ) = standard normal pdf
        phi_xi = norm.pdf(xi)

        # g(u) = [2^β-(2^β-1)u]^(-(1+β)/β) / φ(ξ)
        exponent = -(1 + beta) / beta
        g = np.zeros_like(u)
        valid_mask = (base > 0) & (phi_xi > 1e-300)
        g[valid_mask] = (base[valid_mask] ** exponent) / phi_xi[valid_mask]

        # kernel K(v,w) = min(v,w) - v*w
        def kernel(v, w):
            return np.minimum(v, w) - v * w

        result = 0.0
        for i in range(len(u)):
            for j in range(len(u)):
                if type_num == 1:
                    # Λ_{h,1}: g_i * g_j
                    val = weights[i] * weights[j] * kernel(u[i], u[j]) * g[i] * g[j]
                elif type_num == 2:
                    # Λ_{h,2}: g_i * ξ_j * g_j
                    val = weights[i] * weights[j] * kernel(u[i], u[j]) * g[i] * xi[j] * g[j]
                elif type_num == 3:
                    # Λ_{h,3}: ξ_i * g_i * ξ_j * g_j
                    val = weights[i] * weights[j] * kernel(u[i], u[j]) * xi[i] * g[i] * xi[j] * g[j]
                else:
                    raise ValueError("Invalid type_num")
                result += w[i] * w[j] * val

        return result

    def compute_variance_covariance(self, alpha, beta):
        """
        Compute Σ_L for ETELN (Different h, Same J)
        """
        delta = ((2**beta - 1) / (alpha * beta)) ** 2

        L1 = self.compute_lambda_integral(1, alpha, beta)
        L2 = self.compute_lambda_integral(2, alpha, beta)
        L3 = self.compute_lambda_integral(3, alpha, beta)

        # PLUS signs in off-diagonal (different from ETELL)
        Sigma11 = delta * L1
        Sigma12 = delta * (2*np.log(self.theta)*L1 + 2/alpha*L2)
        Sigma22 = delta * (4*np.log(self.theta)**2*L1 + 8*np.log(self.theta)/alpha*L2 + 4/alpha**2*L3)

        return np.array([[Sigma11, Sigma12], [Sigma12, Sigma22]])

    def fisher_information(self, alpha, beta, n):
        """
        Compute Fisher Information for ETELN
        """
        # Integration over u ∈ [0,1]
        u_int = np.linspace(1e-6, 1 - 1e-6, 64)

        # u_0(β) = [2^β - (2^β-1)u]^(-1/β)
        u0 = (2**beta - (2**beta - 1) * u_int) ** (-1/beta)
        u0 = np.clip(u0, 1e-9, 1 - 1e-9)

        # ξ = Φ⁻¹[u₀]
        xi = norm.ppf(u0)
        phi_xi = norm.pdf(xi)
        Phi_xi = norm.cdf(xi)

        # Avoid division by zero
        ratio = np.where(Phi_xi > 1e-300, phi_xi / Phi_xi, 0.0)
        ratio2 = np.where(Phi_xi > 1e-300, (phi_xi / Phi_xi) ** 2, 0.0)

        # Expectations
        moment2 = np.trapz(xi**2, u_int) / alpha**2
        moment_ratio1 = np.trapz(ratio * xi, u_int) / alpha
        moment_ratio3 = np.trapz(xi**3 * ratio, u_int) / alpha**2
        moment_ratio2_sq = np.trapz(xi**2 * ratio2, u_int) / alpha**2

        # Fisher Information elements
        two_pow_beta = 2**beta
        denom = two_pow_beta - 1

        # I_ββ (same as ETELL)
        Ibb = (1 / beta**2) * (1 - (two_pow_beta * beta**2 * (np.log(2)**2)) / (denom**2))

        # I_αα
        Iaa = (1 / alpha**2) * (1 + moment2 + (beta + 1)**2 * moment_ratio2_sq +
                                 2 * alpha * (beta + 1) * moment_ratio3)

        # I_αβ
        Iab = moment_ratio1

        return n * np.array([[Iaa, Iab], [Iab, Ibb]])

    # ---- ARE ----
    def compute_ARE(self, mu1, mu2, n):
        """Compute ARE for ETELN L-estimator vs MLE"""
        alpha_hat, beta_hat = self.solve_beta(mu1, mu2)

        if np.isnan(alpha_hat) or np.isnan(beta_hat):
            return np.nan

        Sigma = self.compute_variance_covariance(alpha_hat, beta_hat) / n

        # Jacobian (finite differences)
        eps = 1e-6
        def get_params(m1, m2):
            a, b = self.solve_beta(m1, m2)
            if np.isnan(a) or np.isnan(b):
                return alpha_hat, beta_hat
            return a, b

        alpha_p1, beta_p1 = get_params(mu1 + eps, mu2)
        alpha_m1, beta_m1 = get_params(mu1 - eps, mu2)
        alpha_p2, beta_p2 = get_params(mu1, mu2 + eps)
        alpha_m2, beta_m2 = get_params(mu1, mu2 - eps)

        D = np.array([
            [(alpha_p1 - alpha_m1) / (2*eps), (alpha_p2 - alpha_m2) / (2*eps)],
            [(beta_p1 - beta_m1) / (2*eps), (beta_p2 - beta_m2) / (2*eps)]
        ])

        S_K = D @ Sigma @ D.T
        I = self.fisher_information(alpha_hat, beta_hat, n)

        try:
            S_MLE = np.linalg.inv(I)
        except np.linalg.LinAlgError:
            return np.nan

        det_S_K = np.linalg.det(S_K)
        det_S_MLE = np.linalg.det(S_MLE)

        if det_S_K > 0 and det_S_MLE > 0:
            ARE_det = np.sqrt(det_S_MLE / det_S_K)
        else:
            ARE_det = np.nan

        return ARE_det



In [ ]:
# ==============================================================
# ETELN Simulation Study - Same J Design
# ==============================================================

class ETELNSimulation:
    """
    ETELN Simulation with L-estimation (Same J, Different h design)
      - Stable quantile generation via Φ⁻¹
      - Vectorized Λ integrals with 1/φ(ξ) terms
      - Robust beta solver
      - MLE for ETELN
      - Determinant-based RE/ARE
    """

    # ----------------------------- init & quadrature -----------------------------
    def __init__(self, theta=1.0, alpha=2.0, beta=0.5, n_quad=200,
                 use_det_re=True, use_numeric_info=False, rng=None):
        self.theta = float(theta)
        self.alpha_true = float(alpha)
        self.beta_true = float(beta)
        self.use_det_re = bool(use_det_re)
        self.use_numeric_info = bool(use_numeric_info)

        self.nodes, self.weights = leggauss(n_quad)
        self.u = 0.5 * (self.nodes + 1.0)   # map [-1,1] -> [0,1]
        self.w = 0.5 * self.weights
        self._K = None                      # cache Brownian-bridge kernel
        self.rng = np.random.default_rng(rng)

    # ----------------------------- distributions & transforms -----------------------------
    @staticmethod
    def kumaraswamy_weight(u, a, b):
        return a * b * (u**(a-1.0)) * ((1.0 - u**a)**(b-1.0))

    def _stable_terms_eteln(self, beta, u):
        """
        Return u0, xi, phi_xi, g for ETELN:
          u0 = [2^β - (2^β-1)u]^(-1/β)
          xi = Φ⁻¹[u0]
          phi_xi = φ(xi)
          g = [2^β-(2^β-1)u]^(-(1+β)/β) / φ(ξ)
        """
        with np.errstate(all='ignore'):
            two_pow_beta = np.exp(beta * np.log(2.0))
            base = two_pow_beta - (two_pow_beta - 1.0) * u
            base = np.maximum(base, 1e-15)

            # Handle beta ≈ 0 case
            if abs(beta) < 1e-8:
                # Limit: [2^β - (2^β-1)u]^(-1/β) → 2^(-(1-u))
                u0 = np.exp(-(1.0 - u) * np.log(2.0))  # = 2^(-(1-u))
                exponent = -1.0  # limit of -(1+β)/β as β→0
            else:
                u0 = base ** (-1.0/beta)
                exponent = -(1.0 + beta) / beta

            u0 = np.clip(u0, 1e-9, 1 - 1e-9)

            xi = norm.ppf(u0)
            xi = np.where(np.isfinite(xi), xi, 0.0)

            phi_xi = norm.pdf(xi)
            phi_xi = np.maximum(phi_xi, 1e-300)

            g = np.zeros_like(u)
            valid = (base > 0) & (phi_xi > 1e-300)
            g[valid] = (base[valid] ** exponent) / phi_xi[valid]
            g = np.where(np.isfinite(g), g, 0.0)

        return u0, xi, phi_xi, g

    def _kernel_matrix(self):
        if self._K is None:
            u = self.u
            self._K = np.minimum(u[:,None], u[None,:]) - (u[:,None] * u[None,:])
        return self._K

    # ----------------------------- quantile & sampling -----------------------------
    def generate_eteln_sample(self, n):
        """
        Inverse-transform sampling from ETELN(θ, α, β).
        Q(u) = θ exp{(1/α) Φ⁻¹[(2^β-(2^β-1)u)^(-1/β)]}
        """
        u = self.rng.uniform(0.0, 1.0, int(n))

        with np.errstate(all='ignore'):
            if abs(self.beta_true) < 1e-8:
                # Limit case
                u0 = 2.0 ** (-(1.0 - u))
            else:
                two_pow_beta = np.exp(self.beta_true * np.log(2.0))
                base = two_pow_beta - (two_pow_beta - 1.0) * u
                u0 = base ** (-1/self.beta_true)

            u0 = np.clip(u0, 1e-9, 1 - 1e-9)
            xi = norm.ppf(u0)
            x = self.theta * np.exp((1.0/self.alpha_true) * xi)

        return x

    # ----------------------------- d_k and tau -----------------------------
    def compute_dk(self, beta, k, a, b):
        """
        d_k(β) = ∫_0^1 J(u;a,b) [Φ⁻¹[(2^β-(2^β-1)u)^(-1/β)]]^k du
        """
        u, w = self.u, self.w
        J = self.kumaraswamy_weight(u, a, b)

        # Handle beta ≈ 0 separately
        if abs(beta) < 1e-8:
            u0 = np.exp(-(1.0 - u) * np.log(2.0))  # 2^(-(1-u))
            u0 = np.clip(u0, 1e-9, 1 - 1e-9)
            xi = norm.ppf(u0)
        else:
            u0, xi, _, _ = self._stable_terms_eteln(beta, u)

        if k == 1:
            integrand = J * xi
        elif k == 2:
            integrand = J * (xi ** 2)
        else:
            raise ValueError("k must be 1 or 2")

        return np.sum(w * integrand)

    def tau(self, beta, a, b):
        """τ_h(β) = d_{h,2}(β) - [d_{h,1}(β)]²"""
        d1 = self.compute_dk(beta, 1, a, b)
        d2 = self.compute_dk(beta, 2, a, b)
        return d2 - d1**2

    # ----------------------------- Λ integrals & covariance -----------------------------
    def _lambda_triplet(self, alpha, beta, a, b):
        """
        Vectorized Λ1, Λ2, Λ3 with 1/φ(ξ) terms for ETELN
        """
        u, w = self.u, self.w
        J = self.kumaraswamy_weight(u, a, b)
        u0, xi, phi_xi, g = self._stable_terms_eteln(beta, u)

        W = (w * J)[:,None] * (w * J)[None,:]
        K = self._kernel_matrix()

        G = g[:,None] * g[None,:]
        xi_g_right = (xi[None,:] * g[None,:])
        xi_g_xi_g = (xi[:,None] * g[:,None]) * (xi[None,:] * g[None,:])

        L1 = np.sum(W * K * G)
        L2 = np.sum(W * K * (g[:,None] * xi_g_right))
        L3 = np.sum(W * K * xi_g_xi_g)

        return L1, L2, L3

    def Sigma_mu(self, alpha, beta, a, b):
        """
        Asymptotic covariance of (μ̂1, μ̂2) for ETELN
        Note: PLUS signs in off-diagonal (different from ETELL)
        """
        with np.errstate(all='ignore'):
            delta = ((np.exp(beta * np.log(2.0)) - 1.0) / (alpha * beta)) ** 2

        L1, L2, L3 = self._lambda_triplet(alpha, beta, a, b)
        logt = np.log(self.theta)

        # PLUS signs (not minus!)
        S11 = delta * L1
        S12 = delta * (2*logt*L1 + 2/alpha * L2)
        S22 = delta * (4*(logt**2)*L1 + 8*logt/alpha * L2 + 4/(alpha**2)*L3)

        S = np.array([[S11, S12], [S12, S22]])
        return 0.5 * (S + S.T)

    # ----------------------------- solving beta and alpha from sample L-moments -----------------------------
    def solve_beta(self, mu1, mu2, a, b):
        """
        Solve for β via d_{h,1}/sqrt(tau) = (μ1 - log θ)/sqrt(Δ)
        Note: DIFFERENT SIGN from ETELL! (μ1 - log θ) not (log θ - μ1)
        """
        Delta = mu2 - mu1**2
        if Delta <= 1e-12:
            return self.beta_true

        # For ETELN: (μ1 - log θ) [POSITIVE]
        target = (mu1 - np.log(self.theta)) / np.sqrt(Delta)

        def R(beta):
            # Protect against beta = 0
            beta_safe = beta if abs(beta) > 1e-8 else np.sign(beta) * 1e-8

            d1 = self.compute_dk(beta_safe, 1, a, b)
            t = self.tau(beta_safe, a, b)
            if not np.isfinite(d1) or t <= 0:
                return np.nan
            return d1/np.sqrt(t) - target

        grid = np.linspace(-3.5, 3.5, 71)
        vals = np.array([R(bi) for bi in grid])
        sgn = np.sign(vals)

        for i in range(len(grid)-1):
            if np.isfinite(vals[i]) and np.isfinite(vals[i+1]) and sgn[i]*sgn[i+1] < 0:
                try:
                    return brentq(lambda b: R(b), grid[i], grid[i+1], xtol=1e-7, maxiter=100)
                except:
                    continue

        out = minimize_scalar(lambda b: (R(b) if np.isfinite(R(b)) else 1e6)**2,
                              bounds=(-4,4), method='bounded')
        return out.x if out.success else self.beta_true

    # ----------------------------- L-estimator on a sample -----------------------------
    def kumaraswamy_l_estimator(self, x, a, b):
        """L-estimator with J(u;a,b) weights for ETELN"""
        x = np.asarray(x)
        x = x[x >= self.theta]
        n = x.size
        if n < 3:
            return np.nan, np.nan

        xs = np.sort(x)
        i = np.arange(1, n+1)
        uo = i/(n+1.0)
        J = self.kumaraswamy_weight(uo, a, b)

        lx = np.log(xs)
        mu1 = np.mean(J * lx)
        mu2 = np.mean(J * (lx**2))

        Delta = mu2 - mu1**2
        if Delta <= 0:
            return np.nan, np.nan

        beta_hat = self.solve_beta(mu1, mu2, a, b)
        t_beta = self.tau(beta_hat, a, b)

        if not np.isfinite(t_beta) or t_beta <= 0:
            return np.nan, np.nan

        alpha_hat = np.sqrt(t_beta / Delta)
        return alpha_hat, beta_hat

    # ----------------------------- MLE -----------------------------
    def mle_eteln(self, x):
        """
        MLE for ETELN (θ known): maximize ll(α,β)
        """
        xv = np.asarray(x)
        xv = xv[xv >= self.theta]
        n = xv.size
        if n < 5:
            return np.nan, np.nan

        def nll(params):
            alpha, beta = params
            if alpha <= 0 or np.abs(beta) > 5:
                return 1e10
            try:
                with np.errstate(all='ignore'):
                    if abs(beta) < 1e-8:
                        const = -np.log(np.log(2.0))
                    else:
                        two_b = np.exp(beta * np.log(2.0))
                        const = np.log(np.abs(beta)) - np.log(np.abs(two_b - 1.0))

                    # ETELN log-likelihood
                    zi = alpha * np.log(xv / self.theta)
                    phi_zi = norm.pdf(zi)
                    Phi_zi = norm.cdf(zi)

                    # Avoid log(0)
                    Phi_zi = np.maximum(Phi_zi, 1e-300)
                    phi_zi = np.maximum(phi_zi, 1e-300)

                    ll = (n * np.log(alpha) + n * const
                          - n * 0.5 * np.log(2*np.pi)
                          - np.sum(np.log(xv))
                          - 0.5 * alpha**2 * np.sum((np.log(xv/self.theta))**2)
                          - (beta + 1) * np.sum(np.log(Phi_zi)))

                    return -ll if np.isfinite(ll) else 1e10
            except Exception:
                return 1e10

        # Initial guess
        lx = np.log(xv)
        m1 = lx.mean()
        m2 = (lx**2).mean()
        alpha0 = 1.0 / np.sqrt(max(m2 - m1**2, 1e-4))
        beta0 = np.clip(self.beta_true, -3.0, 3.0)

        try:
            res = minimize(nll, x0=[alpha0, beta0],
                          bounds=[(0.05, 10.0), (-3.0, 3.0)],
                          method="L-BFGS-B")
            if res.success and res.fun < 1e9:
                return res.x[0], res.x[1]

            # Fallback
            res = minimize(nll, x0=[self.alpha_true, self.beta_true],
                          bounds=[(0.05, 10.0), (-3.0, 3.0)],
                          method="L-BFGS-B")
            return (res.x[0], res.x[1]) if res.success else (np.nan, np.nan)
        except Exception:
            return np.nan, np.nan

    # ----------------------------- Fisher information -----------------------------
    def fisher_information(self, alpha, beta, n):
        """
        Fisher Information for ETELN using numerical integration
        """
        try:
            u_int = np.linspace(1e-6, 1 - 1e-6, 35)

            with np.errstate(all='ignore'):
                two_b = np.exp(beta * np.log(2.0))
                u0 = (two_b - (two_b - 1.0) * u_int) ** (-1/beta)
                u0 = np.clip(u0, 1e-9, 1 - 1e-9)

                xi = norm.ppf(u0)
                phi_xi = norm.pdf(xi)
                Phi_xi = norm.cdf(xi)

                ratio = np.where(Phi_xi > 1e-300, phi_xi / Phi_xi, 0.0)
                ratio2 = np.where(Phi_xi > 1e-300, (phi_xi / Phi_xi) ** 2, 0.0)

                moment2 = np.trapz(xi**2, u_int) / alpha**2
                moment_ratio1 = np.trapz(ratio * xi, u_int) / alpha
                moment_ratio3 = np.trapz(xi**3 * ratio, u_int) / alpha**2
                moment_ratio2_sq = np.trapz(xi**2 * ratio2, u_int) / alpha**2

                denom = two_b - 1.0
                ln2 = np.log(2.0)

                Ibb = (1/beta**2) * (1 - (two_b * beta**2 * (ln2**2)) / (denom**2))
                Iaa = (1/alpha**2) * (1 + moment2 + (beta + 1)**2 * moment_ratio2_sq +
                                       2 * alpha * (beta + 1) * moment_ratio3)
                Iab = moment_ratio1

                I = n * np.array([[Iaa, Iab], [Iab, Ibb]])
                return I
        except:
            return None

    # ----------------------------- ARE computation -----------------------------
    def _population_mu(self, alpha, beta, a, b):
        """Compute population μ1, μ2 for given (α,β) and Kumaraswamy (a,b)"""
        u, w = self.u, self.w

        with np.errstate(all='ignore'):
            two_b = np.exp(beta * np.log(2.0))
            base = two_b - (two_b - 1.0) * u
            u0 = base ** (-1/beta)
            u0 = np.clip(u0, 1e-9, 1 - 1e-9)
            xi = norm.ppf(u0)

            q = self.theta * np.exp((1.0/alpha) * xi)
            J = self.kumaraswamy_weight(u, a, b)

            mu1 = np.sum(w * J * np.log(q))
            mu2 = np.sum(w * J * (np.log(q)**2))

        return mu1, mu2

    def _jacobian_D(self, mu1, mu2, a, b, alpha_hat, beta_hat):
        """Numeric Jacobian D = ∂(α,β)/∂(μ1,μ2)"""
        eps1 = 1e-6 * max(1.0, abs(mu1))
        eps2 = 1e-6 * max(1.0, abs(mu2))

        def solve_pair(m1, m2):
            try:
                b = self.solve_beta(m1, m2, a, b)
                t = self.tau(b, a, b)
                a_hat = np.sqrt(t / max(m2 - m1**2, 1e-12))
                return a_hat, b
            except Exception:
                return alpha_hat, beta_hat

        a_p, b_p = solve_pair(mu1 + eps1, mu2)
        a_m, b_m = solve_pair(mu1 - eps1, mu2)
        d11 = (a_p - a_m) / (2*eps1)
        d21 = (b_p - b_m) / (2*eps1)

        a_p, b_p = solve_pair(mu1, mu2 + eps2)
        a_m, b_m = solve_pair(mu1, mu2 - eps2)
        d12 = (a_p - a_m) / (2*eps2)
        d22 = (b_p - b_m) / (2*eps2)

        D = np.array([[d11, d12], [d21, d22]])
        return D

    def compute_theoretical_are(self, a, b):
        """Use ETELNARE class to compute theoretical ARE"""

        solver = ETELNARE(theta=self.theta, kuma_a=a, kuma_b=b, n_quad=200)
        mu1, mu2 = self._population_mu(self.alpha_true, self.beta_true, a, b)
        return solver.compute_ARE(mu1, mu2, n=5000)

    # ----------------------------- simulation with RE -----------------------------
    def run_simulation_with_re_se(self, sample_sizes, kuma_params,
                                  n_batches=10, sims_per_batch=100, verbose=True, ref_at="true"):
        """
        Run simulation study with RE computation
        [Same structure as ETELL version]
        """
        all_results = {}
        for n in sample_sizes:
            if verbose:
                print(f"\nRunning n={n} with {n_batches} batches...")

            # Precompute ARE∞ for each (a,b)
            are_inf = {(a,b): self.compute_theoretical_are(a, b) for (a,b) in kuma_params}

            batch_stats = []
            for bidx in range(n_batches):
                if verbose and bidx % 2 == 0:
                    print(f"  Batch {bidx+1}/{n_batches}")

                est = {"MLE": {"alpha": [], "beta": []}}
                for a,b in kuma_params:
                    est[f"K({a},{b})"] = {"alpha": [], "beta": []}

                # Simulations in this batch
                for _ in range(sims_per_batch):
                    x = self.generate_eteln_sample(n)
                    a_mle, b_mle = self.mle_eteln(x)
                    if np.isfinite(a_mle) and np.isfinite(b_mle):
                        est["MLE"]["alpha"].append(a_mle)
                        est["MLE"]["beta"].append(b_mle)
                    for a,b in kuma_params:
                        ak, bk = self.kumaraswamy_l_estimator(x, a, b)
                        if np.isfinite(ak) and np.isfinite(bk):
                            est[f"K({a},{b})"]["alpha"].append(ak)
                            est[f"K({a},{b})"]["beta"].append(bk)

                # Per-batch stats
                batch = {}

                def _winsorize_pair(a_vals, b_vals, p=0.01):
                    ax = np.asarray(a_vals, float)
                    bx = np.asarray(b_vals, float)
                    if ax.size < 3:
                        return ax, bx
                    lo = int(np.floor(p*ax.size))
                    hi = int(np.ceil((1-p)*ax.size))
                    axs = np.sort(ax)
                    bxs = np.sort(bx)
                    a_lo, a_hi = axs[lo], axs[min(hi, ax.size-1)]
                    b_lo, b_hi = bxs[lo], bxs[min(hi, bx.size-1)]
                    ax_cl = np.clip(ax, a_lo, a_hi)
                    bx_cl = np.clip(bx, b_lo, b_hi)
                    return ax_cl, bx_cl

                def det_re(a_list, b_list, S_asymp_mle_ref):
                    vals = np.c_[a_list, b_list]
                    if vals.shape[0] < 3:
                        return np.nan
                    a_vals, b_vals = _winsorize_pair(vals[:,0], vals[:,1])
                    S = np.cov(np.c_[a_vals, b_vals], rowvar=False, ddof=1)
                    S = 0.5*(S + S.T) + 1e-9*np.eye(2)
                    den = np.linalg.det(S)
                    num = np.linalg.det(S_asymp_mle_ref)
                    return np.sqrt(num/den) if den > 0 else np.nan

                # MLE row
                if len(est["MLE"]["alpha"]) > 0:
                    avals = np.array(est["MLE"]["alpha"])
                    bvals = np.array(est["MLE"]["beta"])

                    if ref_at == "batch":
                        alpha_ref = float(np.mean(avals))
                        beta_ref = float(np.mean(bvals))
                    else:
                        alpha_ref = self.alpha_true
                        beta_ref = self.beta_true

                    I_ref = self.fisher_information(alpha_ref, beta_ref, n)
                    if I_ref is None:
                        continue
                    S_asympt_mle_ref = np.linalg.inv(I_ref)

                    batch["MLE"] = {
                        "alpha_mean": np.mean(avals) / self.alpha_true,
                        "alpha_se": np.std(avals, ddof=1) / (self.alpha_true * np.sqrt(len(avals))),
                        "beta_mean": np.mean(bvals) / self.beta_true,
                        "beta_se": np.std(bvals, ddof=1) / (self.beta_true * np.sqrt(len(bvals))),
                        "re": det_re(avals, bvals, S_asympt_mle_ref),
                        "re_asymptotic": 1.0
                    }

                    # K(a,b) rows
                    for a,b in kuma_params:
                        key = f"K({a},{b})"
                        if len(est[key]["alpha"]) > 0:
                            avals_k = np.array(est[key]["alpha"])
                            bvals_k = np.array(est[key]["beta"])
                            batch[key] = {
                                "alpha_mean": np.mean(avals_k) / self.alpha_true,
                                "alpha_se": np.std(avals_k, ddof=1) / (self.alpha_true * np.sqrt(len(avals_k))),
                                "beta_mean": np.mean(bvals_k) / self.beta_true,
                                "beta_se": np.std(bvals_k, ddof=1) / (self.beta_true * np.sqrt(len(bvals_k))),
                                "re": det_re(avals_k, bvals_k, S_asympt_mle_ref),
                                "re_asymptotic": are_inf[(a,b)]
                            }

                batch_stats.append(batch)

            # Aggregate across batches
            final = {}
            keys = set().union(*[b.keys() for b in batch_stats])
            for key in keys:
                def collect(field):
                    vals = [b[key][field] for b in batch_stats
                           if key in b and field in b[key] and np.isfinite(b[key][field])]
                    return np.array(vals)

                a_mean = collect("alpha_mean")
                a_se = collect("alpha_se")
                b_mean = collect("beta_mean")
                b_se = collect("beta_se")
                re_vals = collect("re")
                re_inf = collect("re_asymptotic")

                if a_mean.size > 0:
                    final[key] = {
                        "alpha_mean": a_mean.mean(),
                        "alpha_se": a_se.mean() if a_se.size>0 else np.nan,
                        "beta_mean": b_mean.mean() if b_mean.size>0 else np.nan,
                        "beta_se": b_se.mean() if b_se.size>0 else np.nan,
                        "re": re_vals.mean() if re_vals.size>0 else np.nan,
                        "re_se": re_vals.std(ddof=1)/np.sqrt(re_vals.size) if re_vals.size>1 else np.nan,
                        "re_asymptotic": (1.0 if key=="MLE" else (re_inf.mean() if re_inf.size>0 else np.nan))
                    }

            all_results[n] = final

            if verbose:
                print(f"  Total simulations (MLE): {n_batches * sims_per_batch}")

        return all_results

    # ----------------------------- pretty printer -----------------------------
    def print_results_table(self, results, sample_sizes, kuma_params):
        print("\n" + "="*120)
        print(f"Table: Standardized MEAN and RE from ETELN(α={self.alpha_true}, β={self.beta_true}, θ={self.theta})")
        print("Entries are mean values (standard errors)")
        print("="*120)

        col_w, last_w = 14, 10
        header = "KS Par".ljust(14)
        for n in sample_sizes:
            header += f"{f'n={n}':^{col_w*2}}"
        header += f"{'n→∞':^{last_w*2}}"
        print(header)

        sub = "(a,b)".ljust(14)
        for _ in sample_sizes:
            sub += f"{'α̂/α':>{col_w}}{'β̂/β':>{col_w}}"
        sub += f"{'α̂/α':>{last_w}}{'β̂/β':>{last_w}}"
        print(sub)

        # Means
        print("\nMEAN VALUES:")
        def row_for(key, label=None):
            lab = (label or key).ljust(14)
            out = lab
            for n in sample_sizes:
                if n in results and key in results[n]:
                    s = results[n][key]
                    out += f"{s['alpha_mean']:5.2f}({s['alpha_se']:.3f})".rjust(col_w)
                    out += f"{s['beta_mean']:5.2f}({s['beta_se']:.3f})".rjust(col_w)
                else:
                    out += f"{'---':>{col_w*2}}"
            out += f"{'1.00':>{last_w}}{'1.00':>{last_w}}"
            print(out)

        row_for("MLE", "MLE")
        for a,b in kuma_params:
            row_for(f"K({a},{b})", f"J({a},{b})")

        # RE
        print("\n" + "-"*120)
        print("RELATIVE EFFICIENCY:")
        def re_row(key, label=None):
            lab = (label or key).ljust(14)
            out = lab
            for n in sample_sizes:
                if n in results and key in results[n]:
                    s = results[n][key]
                    re = s.get("re", np.nan)
                    se = s.get("re_se", np.nan)
                    out += f"{re:5.3f}({se:.3f})".rjust(col_w)
                else:
                    out += f"{'---':>{col_w}}"
            # Asymptotic
            n0 = sample_sizes[0]
            if n0 in results and key in results[n0]:
                out += f"{results[n0][key]['re_asymptotic']:5.3f}".rjust(last_w)
            else:
                out += f"{'---':>{last_w}}"
            print(out)

        re_row("MLE", "MLE")
        for a,b in kuma_params:
            re_row(f"K({a},{b})", f"J({a},{b})")


# ----------------------------- example runner -----------------------------
def run_eteln_simulation_study():
    alpha_true, beta_true, theta_true = 2.0, 0.5, 1.0
    sample_sizes = [100, 250, 500, 1000]
    kuma_params = [
        (1.0, 1.0),
        (0.3, 1.0),
        (1.2, 1.3),
        (0.8, 1.0),
        (4.0, 12.0),
        (2.0, 1.0),
        (5.0, 5.0),
    ]

    print("ETELN Simulation Study")
    print(f"True params: α={alpha_true}, β={beta_true}, θ={theta_true}")

    sim = ETELNSimulation(theta=theta_true, alpha=alpha_true, beta=beta_true,
                          n_quad=600, use_det_re=True, use_numeric_info=False, rng=123)

    results = sim.run_simulation_with_re_se(
        sample_sizes=sample_sizes,
        kuma_params=kuma_params,
        n_batches=50,
        sims_per_batch=200,
        verbose=True,
        ref_at="true",
    )

    sim.print_results_table(results, sample_sizes, kuma_params)
    return results


if __name__ == "__main__":
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    t0 = time.perf_counter()
    _ = run_eteln_simulation_study()
    print(f"\n⏱️ Total runtime: {time.perf_counter() - t0:.2f} s")

ETELN Simulation Study
True params: α=2.0, β=0.5, θ=1.0

Running n=100 with 50 batches...
  Batch 1/50
  Batch 3/50
  Batch 5/50
  Batch 7/50
  Batch 9/50
  Batch 11/50
  Batch 13/50
  Batch 15/50
  Batch 17/50
  Batch 19/50
  Batch 21/50
  Batch 23/50
  Batch 25/50
  Batch 27/50
  Batch 29/50
  Batch 31/50
  Batch 33/50
  Batch 35/50
  Batch 37/50
  Batch 39/50
  Batch 41/50
  Batch 43/50
  Batch 45/50
  Batch 47/50
  Batch 49/50
  Total simulations (MLE): 10000

Running n=250 with 50 batches...
  Batch 1/50
  Batch 3/50
  Batch 5/50
  Batch 7/50
  Batch 9/50
  Batch 11/50
  Batch 13/50
  Batch 15/50
  Batch 17/50
  Batch 19/50
  Batch 21/50
  Batch 23/50
  Batch 25/50
  Batch 27/50
  Batch 29/50
  Batch 31/50
  Batch 33/50
  Batch 35/50
  Batch 37/50
  Batch 39/50
  Batch 41/50
  Batch 43/50
  Batch 45/50
  Batch 47/50
  Batch 49/50
  Total simulations (MLE): 10000

Running n=500 with 50 batches...
  Batch 1/50
  Batch 3/50
  Batch 5/50
  Batch 7/50
  Batch 9/50
  Batch 11/50
  Batch

# ==============================================================
# ETELN Simulation Different J
# ==============================================================


In [ ]:
# ==============================================================
# ETELN Simulation - DESIGN B (Different J, Same h)
# ==============================================================

class ETELNSimulation_DesignB:
    """
    Design B for ETELN with enhanced robustness:
      - Different J weights (J₁ ≠ J₂)
      - Same h = log(x)
      - Adaptive sign handling for Δ_w
      - Multiple solving strategies
      - Φ⁻¹ transformations with 1/φ(ξ) terms
    """

    def __init__(self, theta=1.0, alpha=2.0, beta=0.5, n_quad=250,
                 use_det_re=True, use_numeric_info=False, rng=None):
        self.theta = float(theta)
        self.alpha_true = float(alpha)
        self.beta_true = float(beta)
        self.use_det_re = bool(use_det_re)
        self.use_numeric_info = bool(use_numeric_info)

        self.nodes, self.weights = leggauss(n_quad)
        self.u = 0.5 * (self.nodes + 1.0)
        self.w = 0.5 * self.weights
        self._K = None
        self.rng = np.random.default_rng(rng)

    @staticmethod
    def kumaraswamy_weight(u, a, b):
        """Kumaraswamy pdf with numerical stability"""
        with np.errstate(all='ignore'):
            if a > 20 or b > 20:
                result = a * b * np.exp((a-1)*np.log(u) + (b-1)*np.log(1 - u**a))
            else:
                result = a * b * (u ** (a - 1.0)) * ((1.0 - u**a) ** (b - 1.0))
            return np.nan_to_num(result, nan=0.0, posinf=0.0, neginf=0.0)

    def _stable_terms_eteln(self, beta, u):
        """
        Stable computation for ETELN:
        Returns u0, xi, phi_xi, g
        """
        with np.errstate(all='ignore'):
            two_pow_beta = np.exp(beta * np.log(2.0))
            base = two_pow_beta - (two_pow_beta - 1.0) * u
            base = np.maximum(base, 1e-15)

            if abs(beta) < 1e-8:
                u0 = np.exp(-(1.0 - u) * np.log(2.0))
                exponent = -1.0
            else:
                u0 = base ** (-1.0/beta)
                exponent = -(1.0 + beta) / beta

            u0 = np.clip(u0, 1e-9, 1 - 1e-9)

            xi = norm.ppf(u0)
            xi = np.where(np.isfinite(xi), xi, 0.0)

            phi_xi = norm.pdf(xi)
            phi_xi = np.maximum(phi_xi, 1e-300)

            g = np.zeros_like(u)
            valid = (base > 0) & (phi_xi > 1e-300)
            g[valid] = (base[valid] ** exponent) / phi_xi[valid]
            g = np.where(np.isfinite(g), g, 0.0)

        return u0, xi, phi_xi, g

    def _kernel_matrix(self):
        if self._K is None:
            u = self.u
            self._K = np.minimum(u[:, None], u[None, :]) - (u[:, None] * u[None, :])
        return self._K

    def generate_eteln_sample(self, n):
        """Inverse-transform sampling for ETELN"""
        u = self.rng.uniform(0.0, 1.0, int(n))

        with np.errstate(all='ignore'):
            if abs(self.beta_true) < 1e-8:
                u0 = 2.0 ** (-(1.0 - u))
            else:
                two_pow_beta = np.exp(self.beta_true * np.log(2.0))
                base = two_pow_beta - (two_pow_beta - 1.0) * u
                u0 = base ** (-1/self.beta_true)

            u0 = np.clip(u0, 1e-9, 1 - 1e-9)
            xi = norm.ppf(u0)
            x = self.theta * np.exp((1.0/self.alpha_true) * xi)

        return x

    def compute_dw_k(self, beta, k, a, b):
        """
        d_{w,k}(β) = ∫_0^1 J(u;a,b) [Φ⁻¹[(2^β-(2^β-1)u)^(-1/β)]]^k du
        """
        u, w = self.u, self.w
        J = self.kumaraswamy_weight(u, a, b)

        if abs(beta) < 1e-8:
            u0 = np.exp(-(1.0 - u) * np.log(2.0))
            u0 = np.clip(u0, 1e-9, 1 - 1e-9)
            xi = norm.ppf(u0)
        else:
            u0, xi, _, _ = self._stable_terms_eteln(beta, u)

        xi = np.where(np.isfinite(xi), xi, 0.0)
        integrand = J * xi
        result = np.sum(w * integrand)

        return result if np.isfinite(result) else np.nan

    def tau_w(self, beta, a1, b1, a2, b2):
        """
        τ_w(β) = d_{w,2}(β) - d_{w,1}(β)
        Note: d_{w,1} uses J₁, d_{w,2} uses J₂
        """
        d1 = self.compute_dw_k(beta, 1, a1, b1)
        d2 = self.compute_dw_k(beta, 2, a2, b2)

        if not (np.isfinite(d1) and np.isfinite(d2)):
            return np.nan

        return d2 - d1

    def solve_beta_designB_robust(self, mu1, mu2, a1, b1, a2, b2, verbose=False):
        """
        Ultra-robust β solver for ETELN Design B
        Ψ(β) = d_{w,1}(β)/τ_w(β) - (μ₁ - log θ)/Δ_w = 0
        Note: POSITIVE ratio for ETELN! (different from ETELL)
        """
        Delta_w = mu2 - mu1

        if abs(Delta_w) < 1e-12:
            if verbose:
                print(f"    Δ_w too small: {Delta_w:.2e}")
            return np.nan, False

        def Psi(beta, swap=False):
            """
            For ETELN: Ψ(β) = d_{w,1}/τ_w - (μ₁ - log θ)/Δ_w
            If swap=True, use d_{w,2} instead
            """
            try:
                if swap:
                    d_num = self.compute_dw_k(beta, 2, a2, b2)
                    tau = -self.tau_w(beta, a1, b1, a2, b2)
                else:
                    d_num = self.compute_dw_k(beta, 1, a1, b1)
                    tau = self.tau_w(beta, a1, b1, a2, b2)

                if not (np.isfinite(d_num) and np.isfinite(tau)):
                    return np.nan

                if abs(tau) < 1e-14:
                    return 1e10 * np.sign(tau + 1e-15)

                # ETELN: (μ₁ - log θ) not (log θ - μ₁)
                result = (d_num / tau) - (mu1 - np.log(self.theta)) / Delta_w
                return result if np.isfinite(result) else np.nan
            except:
                return np.nan

        # Multiple solving strategies
        strategies = [
            ("standard", False, (-3.5, 3.5, 150)),
            ("standard", False, (-4.5, 4.5, 200)),
            ("swapped", True, (-3.5, 3.5, 150)),
            ("swapped", True, (-4.5, 4.5, 200)),
        ]

        for strategy_name, swap, (beta_min, beta_max, n_grid) in strategies:
            try:
                grid = np.linspace(beta_min, beta_max, n_grid)
                psi_vals = np.array([Psi(b, swap=swap) for b in grid])

                valid = np.isfinite(psi_vals)
                if np.sum(valid) < 3:
                    continue

                psi_valid = psi_vals[valid]
                beta_valid = grid[valid]

                signs = np.sign(psi_valid)
                sign_changes = np.where(np.diff(signs) != 0)[0]

                if len(sign_changes) == 0:
                    continue

                for idx in sign_changes:
                    a, b = beta_valid[idx], beta_valid[idx + 1]

                    fa, fb = Psi(a, swap=swap), Psi(b, swap=swap)
                    if not (np.isfinite(fa) and np.isfinite(fb)):
                        continue

                    if fa * fb < 0:
                        try:
                            beta_hat = brentq(lambda bb: Psi(bb, swap=swap),
                                            a, b, xtol=1e-10, rtol=1e-9, maxiter=500)

                            tau = self.tau_w(beta_hat, a1, b1, a2, b2)
                            if swap:
                                tau = -tau

                            # POSITIVE ratio for ETELN (not negative!)
                            alpha_hat = tau / Delta_w

                            if (alpha_hat > 0.01 and alpha_hat < 100 and
                                np.isfinite(alpha_hat) and abs(beta_hat) < 10):

                                if verbose and swap:
                                    print(f"    Success with {strategy_name} strategy")

                                return beta_hat, swap
                        except:
                            continue
            except:
                continue

        # Optimization fallback
        for swap in [False, True]:
            try:
                def objective(beta):
                    val = Psi(beta, swap=swap)
                    return abs(val) if np.isfinite(val) else 1e10

                result = minimize_scalar(objective, bounds=(-5, 5), method='bounded',
                                       options={'maxiter': 500})

                if result.success and result.fun < 0.01:
                    beta_hat = result.x
                    tau = self.tau_w(beta_hat, a1, b1, a2, b2)
                    if swap:
                        tau = -tau

                    alpha_hat = tau / Delta_w

                    if alpha_hat > 0.01 and np.isfinite(alpha_hat):
                        if verbose and swap:
                            print(f"    Success with optimization {['standard','swapped'][swap]}")
                        return beta_hat, swap
            except:
                continue

        if verbose:
            print(f"    All strategies failed")

        return np.nan, False

    def kumaraswamy_l_estimator_designB_robust(self, x, a1, b1, a2, b2, verbose=False):
        """L-estimator for ETELN Design B with robust solving"""
        x = np.asarray(x)
        x = x[x >= self.theta]
        n = x.size
        if n < 3:
            return np.nan, np.nan

        xs = np.sort(x)
        i = np.arange(1, n + 1)
        uo = i / (n + 1.0)

        J1 = self.kumaraswamy_weight(uo, a1, b1)
        J2 = self.kumaraswamy_weight(uo, a2, b2)

        lx = np.log(xs)
        mu1_orig = np.mean(J1 * lx)
        mu2_orig = np.mean(J2 * lx)

        Delta_w_orig = mu2_orig - mu1_orig

        if abs(Delta_w_orig) < 1e-12:
            if verbose:
                print(f"  J₁({a1},{b1})×J₂({a2},{b2}): Δ_w ≈ 0")
            return np.nan, np.nan

        beta_hat, swapped = self.solve_beta_designB_robust(
            mu1_orig, mu2_orig, a1, b1, a2, b2, verbose=verbose
        )

        if np.isnan(beta_hat):
            if verbose:
                print(f"  J₁({a1},{b1})×J₂({a2},{b2}): β solve failed")
            return np.nan, np.nan

        tau = self.tau_w(beta_hat, a1, b1, a2, b2)
        if swapped:
            tau = -tau

        if not np.isfinite(tau) or abs(tau) < 1e-12:
            if verbose:
                print(f"  J₁({a1},{b1})×J₂({a2},{b2}): τ_w invalid")
            return np.nan, np.nan

        # POSITIVE ratio for ETELN
        alpha_hat = tau / Delta_w_orig

        if alpha_hat <= 0:
            if verbose:
                print(f"  J₁({a1},{b1})×J₂({a2},{b2}): α̂={alpha_hat:.3f} ≤ 0")
            return np.nan, np.nan

        if not np.isfinite(alpha_hat):
            if verbose:
                print(f"  J₁({a1},{b1})×J₂({a2},{b2}): α̂ not finite")
            return np.nan, np.nan

        # Sanity check
        if (abs(alpha_hat - self.alpha_true) > 5 * self.alpha_true or
            abs(beta_hat - self.beta_true) > 10):
            if verbose:
                print(f"  J₁({a1},{b1})×J₂({a2},{b2}): estimates too far from truth")
            return np.nan, np.nan

        return alpha_hat, beta_hat

    def _lambda_w_pair(self, alpha, beta, ai, bi, aj, bj):
        """
        Compute Λ_{w,ij} with 1/φ(ξ) terms for ETELN
        """
        u, w = self.u, self.w
        Ji = self.kumaraswamy_weight(u, ai, bi)
        Jj = self.kumaraswamy_weight(u, aj, bj)
        u0, xi, phi_xi, g = self._stable_terms_eteln(beta, u)

        W = (w * Ji)[:, None] * (w * Jj)[None, :]
        K = self._kernel_matrix()
        G = g[:, None] * g[None, :]

        return np.sum(W * K * G)

    def Sigma_mu_designB(self, alpha, beta, a1, b1, a2, b2):
        """Asymptotic covariance for ETELN Design B"""
        delta_sq = ((np.exp(beta * np.log(2.0)) - 1.0) / (alpha * beta)) ** 2

        L11 = self._lambda_w_pair(alpha, beta, a1, b1, a1, b1)
        L12 = self._lambda_w_pair(alpha, beta, a1, b1, a2, b2)
        L22 = self._lambda_w_pair(alpha, beta, a2, b2, a2, b2)

        S = delta_sq * np.array([[L11, L12], [L12, L22]])
        return 0.5 * (S + S.T)

    def mle_eteln(self, x):
        """MLE for ETELN"""
        xv = np.asarray(x)
        xv = xv[xv >= self.theta]
        n = xv.size
        if n < 5:
            return np.nan, np.nan

        def nll(params):
            alpha, beta = params
            if alpha <= 0 or np.abs(beta) > 5:
                return 1e10
            try:
                with np.errstate(all='ignore'):
                    if abs(beta) < 1e-8:
                        const = -np.log(np.log(2.0))
                    else:
                        two_b = np.exp(beta * np.log(2.0))
                        const = np.log(np.abs(beta)) - np.log(np.abs(two_b - 1.0))

                    zi = alpha * np.log(xv / self.theta)
                    phi_zi = norm.pdf(zi)
                    Phi_zi = norm.cdf(zi)

                    Phi_zi = np.maximum(Phi_zi, 1e-300)
                    phi_zi = np.maximum(phi_zi, 1e-300)

                    ll = (n * np.log(alpha) + n * const
                          - n * 0.5 * np.log(2*np.pi)
                          - np.sum(np.log(xv))
                          - 0.5 * alpha**2 * np.sum((np.log(xv/self.theta))**2)
                          - (beta + 1) * np.sum(np.log(Phi_zi)))

                    return -ll if np.isfinite(ll) else 1e10
            except:
                return 1e10

        lx = np.log(xv)
        m1 = lx.mean()
        m2 = (lx**2).mean()
        alpha0 = 1.0 / np.sqrt(max(m2 - m1**2, 1e-4))
        beta0 = np.clip(self.beta_true, -3.0, 3.0)

        try:
            res = minimize(nll, x0=[alpha0, beta0],
                          bounds=[(0.05, 10.0), (-3.0, 3.0)],
                          method="L-BFGS-B")
            if res.success and res.fun < 1e9:
                return res.x[0], res.x[1]

            res = minimize(nll, x0=[self.alpha_true, self.beta_true],
                          bounds=[(0.05, 10.0), (-3.0, 3.0)],
                          method="L-BFGS-B")
            return (res.x[0], res.x[1]) if res.success else (np.nan, np.nan)
        except:
            return np.nan, np.nan

    def fisher_information(self, alpha, beta, n):
        """Fisher Information for ETELN"""
        try:
            u_int = np.linspace(1e-6, 1 - 1e-6, 50)

            with np.errstate(all='ignore'):
                two_b = np.exp(beta * np.log(2.0))
                u0 = (two_b - (two_b - 1.0) * u_int) ** (-1/beta)
                u0 = np.clip(u0, 1e-9, 1 - 1e-9)

                xi = norm.ppf(u0)
                phi_xi = norm.pdf(xi)
                Phi_xi = norm.cdf(xi)

                ratio = np.where(Phi_xi > 1e-300, phi_xi / Phi_xi, 0.0)
                ratio2 = np.where(Phi_xi > 1e-300, (phi_xi / Phi_xi) ** 2, 0.0)

                moment2 = np.trapz(xi**2, u_int) / alpha**2
                moment_ratio1 = np.trapz(ratio * xi, u_int) / alpha
                moment_ratio3 = np.trapz(xi**3 * ratio, u_int) / alpha**2
                moment_ratio2_sq = np.trapz(xi**2 * ratio2, u_int) / alpha**2

                denom = two_b - 1.0
                ln2 = np.log(2.0)

                Ibb = (1/beta**2) * (1 - (two_b * beta**2 * (ln2**2)) / (denom**2))
                Iaa = (1/alpha**2) * (1 + moment2 + (beta + 1)**2 * moment_ratio2_sq +
                                       2 * alpha * (beta + 1) * moment_ratio3)
                Iab = moment_ratio1

                I = n * np.array([[Iaa, Iab], [Iab, Ibb]])
                return I
        except:
            return None

    def compute_theoretical_are_designB(self, a1, b1, a2, b2):
        """Compute theoretical ARE for ETELN Design B"""
        u, w = self.u, self.w

        with np.errstate(all='ignore'):
            two_b = np.exp(self.beta_true * np.log(2.0))
            base = two_b - (two_b - 1.0) * u
            u0 = base ** (-1/self.beta_true)
            u0 = np.clip(u0, 1e-9, 1 - 1e-9)
            xi = norm.ppf(u0)

            q = self.theta * np.exp((1.0/self.alpha_true) * xi)

        J1 = self.kumaraswamy_weight(u, a1, b1)
        J2 = self.kumaraswamy_weight(u, a2, b2)

        mu1 = np.sum(w * J1 * np.log(q))
        mu2 = np.sum(w * J2 * np.log(q))

        beta_hat, swapped = self.solve_beta_designB_robust(mu1, mu2, a1, b1, a2, b2)

        if np.isnan(beta_hat):
            return np.nan

        tau = self.tau_w(beta_hat, a1, b1, a2, b2)
        if swapped:
            tau = -tau

        alpha_hat = tau / (mu2 - mu1)

        if not (np.isfinite(alpha_hat) and np.isfinite(beta_hat) and alpha_hat > 0):
            return np.nan

        try:
            n_large = 5000
            Sigma_mu = self.Sigma_mu_designB(alpha_hat, beta_hat, a1, b1, a2, b2) / n_large

            eps = 1e-6

            def solve_pair(m1, m2):
                try:
                    b, sw = self.solve_beta_designB_robust(m1, m2, a1, b1, a2, b2)
                    tau = self.tau_w(b, a1, b1, a2, b2)
                    if sw:
                        tau = -tau
                    a = tau / (m2 - m1)
                    return a, b
                except:
                    return alpha_hat, beta_hat

            a_p1, b_p1 = solve_pair(mu1 + eps, mu2)
            a_m1, b_m1 = solve_pair(mu1 - eps, mu2)
            a_p2, b_p2 = solve_pair(mu1, mu2 + eps)
            a_m2, b_m2 = solve_pair(mu1, mu2 - eps)

            D = np.array([
                [(a_p1 - a_m1) / (2 * eps), (a_p2 - a_m2) / (2 * eps)],
                [(b_p1 - b_m1) / (2 * eps), (b_p2 - b_m2) / (2 * eps)]
            ])

            if abs(np.linalg.det(D)) < 1e-12:
                return np.nan

            S_L = D @ Sigma_mu @ D.T
            I = self.fisher_information(alpha_hat, beta_hat, n_large)
            if I is None:
                return np.nan

            S_MLE = np.linalg.inv(I)

            det_S_L = np.linalg.det(S_L)
            det_S_MLE = np.linalg.det(S_MLE)

            if det_S_L <= 0 or det_S_MLE <= 0:
                return np.nan

            ARE = np.sqrt(det_S_MLE / det_S_L)

            if not np.isfinite(ARE) or ARE < 1e-6 or ARE > 2:
                return np.nan

            return ARE
        except:
            return np.nan

    def run_simulation_with_re_se_designB(self, sample_sizes, j_pairs,
                                          n_batches=10, sims_per_batch=100,
                                          verbose=True, ref_at="true"):
        """Run simulation for ETELN Design B"""
        all_results = {}

        for n in sample_sizes:
            if verbose:
                print(f"\nRunning n={n} with {n_batches} batches...")

            are_inf = {}
            for (a1, b1), (a2, b2) in j_pairs:
                are = self.compute_theoretical_are_designB(a1, b1, a2, b2)
                are_inf[((a1, b1), (a2, b2))] = are

            batch_stats = []

            for bidx in range(n_batches):
                if verbose and bidx % 5 == 0:
                    print(f"  Batch {bidx + 1}/{n_batches}")

                est = {"MLE": {"alpha": [], "beta": []}}
                for (a1, b1), (a2, b2) in j_pairs:
                    key = f"J1({a1},{b1})×J2({a2},{b2})"
                    est[key] = {"alpha": [], "beta": []}

                for _ in range(sims_per_batch):
                    x = self.generate_eteln_sample(n)

                    a_mle, b_mle = self.mle_eteln(x)
                    if np.isfinite(a_mle) and np.isfinite(b_mle):
                        est["MLE"]["alpha"].append(a_mle)
                        est["MLE"]["beta"].append(b_mle)

                    for (a1, b1), (a2, b2) in j_pairs:
                        key = f"J1({a1},{b1})×J2({a2},{b2})"
                        ak, bk = self.kumaraswamy_l_estimator_designB_robust(
                            x, a1, b1, a2, b2, verbose=False
                        )
                        if np.isfinite(ak) and np.isfinite(bk):
                            est[key]["alpha"].append(ak)
                            est[key]["beta"].append(bk)

                batch = {}

                def _winsorize_pair(a_vals, b_vals, p=0.00):
                    ax = np.asarray(a_vals, float)
                    bx = np.asarray(b_vals, float)
                    if ax.size < 3:
                        return ax, bx
                    lo = int(np.floor(p * ax.size))
                    hi = int(np.ceil((1 - p) * ax.size))
                    axs = np.sort(ax)
                    bxs = np.sort(bx)
                    a_lo, a_hi = axs[lo], axs[min(hi, ax.size - 1)]
                    b_lo, b_hi = bxs[lo], bxs[min(hi, bx.size - 1)]
                    ax_cl = np.clip(ax, a_lo, a_hi)
                    bx_cl = np.clip(bx, b_lo, b_hi)
                    return ax_cl, bx_cl

                def det_re(a_list, b_list, S_asymp_mle_ref):
                    vals = np.c_[a_list, b_list]
                    if vals.shape[0] < 3:
                        return np.nan
                    a_vals, b_vals = _winsorize_pair(vals[:, 0], vals[:, 1])
                    S = np.cov(np.c_[a_vals, b_vals], rowvar=False, ddof=1)
                    S = 0.5 * (S + S.T) + 1e-9 * np.eye(2)
                    den = np.linalg.det(S)
                    num = np.linalg.det(S_asymp_mle_ref)
                    return np.sqrt(num / den) if den > 0 else np.nan

                if len(est["MLE"]["alpha"]) > 0:
                    avals = np.array(est["MLE"]["alpha"])
                    bvals = np.array(est["MLE"]["beta"])

                    if ref_at == "batch":
                        alpha_ref = float(np.mean(avals))
                        beta_ref = float(np.mean(bvals))
                    else:
                        alpha_ref = self.alpha_true
                        beta_ref = self.beta_true

                    I_ref = self.fisher_information(alpha_ref, beta_ref, n)
                    if I_ref is None:
                        continue
                    S_asymp_mle_ref = np.linalg.inv(I_ref)

                    batch["MLE"] = {
                        "alpha_mean": np.mean(avals) / self.alpha_true,
                        "alpha_se": np.std(avals, ddof=1) / (self.alpha_true * np.sqrt(len(avals))),
                        "beta_mean": np.mean(bvals) / self.beta_true,
                        "beta_se": np.std(bvals, ddof=1) / (self.beta_true * np.sqrt(len(bvals))),
                        "re": det_re(avals, bvals, S_asymp_mle_ref),
                        "re_asymptotic": 1.0
                    }

                    for (a1, b1), (a2, b2) in j_pairs:
                        key = f"J1({a1},{b1})×J2({a2},{b2})"
                        if len(est[key]["alpha"]) > 0:
                            avals_k = np.array(est[key]["alpha"])
                            bvals_k = np.array(est[key]["beta"])
                            batch[key] = {
                                "alpha_mean": np.mean(avals_k) / self.alpha_true,
                                "alpha_se": np.std(avals_k, ddof=1) / (self.alpha_true * np.sqrt(len(avals_k))),
                                "beta_mean": np.mean(bvals_k) / self.beta_true,
                                "beta_se": np.std(bvals_k, ddof=1) / (self.beta_true * np.sqrt(len(bvals_k))),
                                "re": det_re(avals_k, bvals_k, S_asymp_mle_ref),
                                "re_asymptotic": are_inf[((a1, b1), (a2, b2))]
                            }

                batch_stats.append(batch)

            final = {}
            keys = set().union(*[b.keys() for b in batch_stats])

            for key in keys:
                def collect(field):
                    vals = [b[key][field] for b in batch_stats
                           if key in b and field in b[key] and np.isfinite(b[key][field])]
                    return np.array(vals)

                a_mean = collect("alpha_mean")
                a_se = collect("alpha_se")
                b_mean = collect("beta_mean")
                b_se = collect("beta_se")
                re_vals = collect("re")
                re_inf = collect("re_asymptotic")

                if a_mean.size > 0:
                    final[key] = {
                        "alpha_mean": a_mean.mean(),
                        "alpha_se": a_se.mean() if a_se.size > 0 else np.nan,
                        "beta_mean": b_mean.mean() if b_mean.size > 0 else np.nan,
                        "beta_se": b_se.mean() if b_se.size > 0 else np.nan,
                        "re": re_vals.mean() if re_vals.size > 0 else np.nan,
                        "re_se": re_vals.std(ddof=1) / np.sqrt(re_vals.size) if re_vals.size > 1 else np.nan,
                        "re_asymptotic": (1.0 if key == "MLE" else (re_inf.mean() if re_inf.size > 0 else np.nan))
                    }

            all_results[n] = final

        return all_results

    def print_results_table_designB(self, results, sample_sizes, j_pairs):
        """Print results table for ETELN Design B"""
        print("\n" + "=" * 140)
        print(f"Design B: Standardized MEAN and RE from ETELN(α={self.alpha_true}, β={self.beta_true}, θ={self.theta})")
        print("Different J (J₁ ≠ J₂), Same h = log(x)")
        print("=" * 140)

        col_w, last_w = 14, 10
        header = "Weight Config".ljust(30)
        for n in sample_sizes:
            header += f"{f'n={n}':^{col_w * 2}}"
        header += f"{'n→∞':^{last_w * 2}}"
        print(header)

        sub = "J₁(a₁,b₁)×J₂(a₂,b₂)".ljust(30)
        for _ in sample_sizes:
            sub += f"{'α̂/α':>{col_w}}{'β̂/β':>{col_w}}"
        sub += f"{'α̂/α':>{last_w}}{'β̂/β':>{last_w}}"
        print(sub)

        print("\nMEAN VALUES:")

        def row_for(key, label=None):
            lab = (label or key).ljust(30)
            out = lab
            for n in sample_sizes:
                if n in results and key in results[n]:
                    s = results[n][key]
                    out += f"{s['alpha_mean']:5.2f}({s['alpha_se']:.3f})".rjust(col_w)
                    out += f"{s['beta_mean']:5.2f}({s['beta_se']:.3f})".rjust(col_w)
                else:
                    out += f"{'---':>{col_w * 2}}"
            out += f"{'1.00':>{last_w}}{'1.00':>{last_w}}"
            print(out)

        row_for("MLE", "MLE")
        for (a1, b1), (a2, b2) in j_pairs:
            key = f"J1({a1},{b1})×J2({a2},{b2})"
            row_for(key, f"J₁({a1},{b1})×J₂({a2},{b2})")

        print("\n" + "-" * 140)
        print("RELATIVE EFFICIENCY:")

        def re_row(key, label=None):
            lab = (label or key).ljust(30)
            out = lab
            for n in sample_sizes:
                if n in results and key in results[n]:
                    s = results[n][key]
                    re = s.get("re", np.nan)
                    se = s.get("re_se", np.nan)
                    out += f"{re:5.3f}({se:.3f})".rjust(col_w)
                else:
                    out += f"{'---':>{col_w}}"

            n0 = sample_sizes[0]
            if n0 in results and key in results[n0]:
                out += f"{results[n0][key]['re_asymptotic']:5.3f}".rjust(last_w)
            else:
                out += f"{'---':>{last_w}}"
            print(out)

        re_row("MLE", "MLE")
        for (a1, b1), (a2, b2) in j_pairs:
            key = f"J1({a1},{b1})×J2({a2},{b2})"
            re_row(key, f"J₁({a1},{b1})×J₂({a2},{b2})")


# ==============================================================
# Runner
# ==============================================================

def run_eteln_simulation_study_designB_robust():
    """Run ETELN Design B simulation"""
    alpha_true, beta_true, theta_true = 2.0, 0.5, 1.0
    sample_sizes = [100, 250, 500, 1000]

    j_pairs = [
        ((1.0, 1.0), (1.0, 2.0)),
        ((1.0, 1.0), (0.3, 1.0)),
        ((1.0, 1.0), (1.2, 1.8)),
        ((1.0, 1.0), (0.8, 1.0)),
        ((1.0, 1.0), (4.0, 12.0)),
        ((1.0, 1.0), (2.0, 1.0)),
    ]

    print("="*70)
    print("ETELN Simulation Study - DESIGN B")
    print("Different J (J₁ ≠ J₂), Same h = log(x)")
    print(f"True params: α={alpha_true}, β={beta_true}, θ={theta_true}")
    print("="*70)

    sim = ETELNSimulation_DesignB(
        theta=theta_true,
        alpha=alpha_true,
        beta=beta_true,
        n_quad=250,
        use_det_re=True,
        use_numeric_info=False,
        rng=123
    )

    results = sim.run_simulation_with_re_se_designB(
        sample_sizes=sample_sizes,
        j_pairs=j_pairs,
        n_batches=50,
        sims_per_batch=200,
        verbose=True,
        ref_at="true"
    )

    sim.print_results_table_designB(results, sample_sizes, j_pairs)
    return results


if __name__ == "__main__":
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    t0 = time.perf_counter()
    results = run_eteln_simulation_study_designB_robust()
    print(f"\n⏱️ Total runtime: {time.perf_counter() - t0:.2f} s")

ETELN Simulation Study - DESIGN B
Different J (J₁ ≠ J₂), Same h = log(x)
True params: α=2.0, β=0.5, θ=1.0

Running n=100 with 50 batches...
  Batch 1/50
  Batch 6/50
  Batch 11/50
  Batch 16/50
  Batch 21/50
  Batch 26/50
  Batch 31/50
  Batch 36/50
  Batch 41/50
  Batch 46/50

Running n=250 with 50 batches...
  Batch 1/50
  Batch 6/50
  Batch 11/50
  Batch 16/50
  Batch 21/50
  Batch 26/50
  Batch 31/50
  Batch 36/50
  Batch 41/50
  Batch 46/50

Running n=500 with 50 batches...
  Batch 1/50
  Batch 6/50
  Batch 11/50
  Batch 16/50
  Batch 21/50
  Batch 26/50
  Batch 31/50
  Batch 36/50
  Batch 41/50
  Batch 46/50

Running n=1000 with 50 batches...
  Batch 1/50
  Batch 6/50
  Batch 11/50
  Batch 16/50
  Batch 21/50
  Batch 26/50
  Batch 31/50
  Batch 36/50
  Batch 41/50
  Batch 46/50

Design B: Standardized MEAN and RE from ETELN(α=2.0, β=0.5, θ=1.0)
Different J (J₁ ≠ J₂), Same h = log(x)
Weight Config                            n=100                       n=250                       n